In [1]:
import pandas as pd
from pathlib import Path
path = Path('/mnt/research-live/equity/reference/equity_data/sa_research/current/')
import pandas as pd 
import numpy as np 
import faiss 
import re
import tqdm
import torch 
import torch.nn.functional as F
import dill as pickle
import datetime
import plotly.express as px
from laserembeddings import Laser
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import copy
import math
from scipy.signal import find_peaks
import seaborn as sns
from pandas.tseries.offsets import BDay
from sklearn.metrics import silhouette_score
from sklearn import metrics
from time import time
from sklearn.cluster import KMeans

/home/yzhong/conda/envs/yihan_gpu/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [198]:
from pathlib import Path
path = Path('/mnt/research-live/equity/reference/equity_data/sa_research/current/')
ead_ = pd.read_parquet(path /'CFM_AMER/equity_data/eqt_earning_announcement_dates.parquet') # date for earnings
news_volume_ =pd.read_parquet(path /'CFM_AMER/equity_data/eqt_bb_news_volumes.parquet') # volume for each date

In [230]:
code='CMG_UN'

In [231]:
# Convert the rows into columns for the eqt_code earning date
ead = ead_.assign(one=1) \
          .rename(columns={'earning_date': 'BDATE'}) \
          .set_index(['BDATE', 'eqt_code']) \
          .squeeze() \
          .unstack()
news_volume = news_volume_.rename(columns={'daclose': 'BDATE'}) \
                         .groupby(['BDATE', 'eqt_code'])['total_count'] \
                         .mean() \
                         .unstack()

In [232]:
# Normalize the news_volume DataFrame by dividing each element by its corresponding rolling mean
df = news_volume.pipe(
    lambda x: x.div(
        x.rolling(window=126, min_periods=42).mean()  # Calculate the rolling mean for each element
    )
)[[code]]

In [233]:
df2 =ead.reindex_like(df)
earning_days = df2.dropna().index.to_list()

In [235]:
def identify_market_on_close(headline):
    pattern = r'^MARKET ON CLOSE.*'
    return re.match(pattern, headline) is not None

In [236]:
def is_all_uppercase(headline):
    # Remove any non-alphabet characters from the headline
    headline_alpha = re.sub(r'[^A-Za-z]', '', headline)
    return headline_alpha.isupper()

Deduplications

In [237]:
import copy

def find_duplicates(df):
    df_copy = copy.deepcopy(df)
    df_copy['headline_lower'] = df_copy['headline'].str.lower()  # Convert headlines to lowercase
    regex_pattern =r'correct:|\(\d+\)\.|update\d+|\(correct\)'
    duplicates = df_copy[df_copy.duplicated(subset='headline_lower', keep='first') | df_copy['headline_lower'].str.contains(regex_pattern)]
    return duplicates.index.tolist()

In [238]:
df_2015=pd.read_parquet('/mnt/research-live/user/yzhong/bloomberg_news_english_single_stock.parquet')
df_2015_mcd=df_2015[df_2015.eqt_code==code]

In [242]:
df_2015_mcd['date']=pd.to_datetime(df_2015_mcd['date'])

/opt/tmp/tmp/ipykernel_360566/3191464300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2015_mcd['date']=pd.to_datetime(df_2015_mcd['date'])


In [245]:
with open('/mnt/research-live/user/yzhong/CMG_UN.txt', 'w') as f:
    for row in filtered.itertuples():
        f.write(str(row.headline) + '\n')

In [244]:
filtered=df_2015_mcd[df_2015_mcd['date'].isin(earning_days)]

In [219]:
df_2022=pd.read_parquet('/mnt/research-live/user/yzhong/bloomberg_news_english_single_stock_topic_2022.parquet')
df_2022_mcd=df_2022[df_2022.eqt_code=='MCD_UN']
filtered_df = df_2022_mcd.loc[(df_2022_mcd['date'] == pd.to_datetime('2022-01-27')) |
                              (df_2022_mcd['date'] == pd.to_datetime('2022-04-28')) |
                              (df_2022_mcd['date'] == pd.to_datetime('2022-07-26')) |
                              (df_2022_mcd['date'] == pd.to_datetime('2022-10-27'))]

filtered_df = filtered_df[~filtered_df['headline'].apply(identify_market_on_close)].reset_index(drop=True)
filtered_df = filtered_df[filtered_df['headline'].apply(is_all_uppercase)].reset_index(drop=True)
filtered_df.drop(index=find_duplicates(filtered_df),inplace=True)

In [220]:
topics_list = list(set(filtered_df['topics'].str.split(' ').sum()))
len(topics_list)

131

Check if there are news that have the same topics

In [221]:
from itertools import combinations

# Extract the topics from the 'topics' column and split them into lists
topics_lists = filtered_df['topics'].str.split(' ').tolist()

# Create a dictionary to store the indices for each unique topic set
topic_indices = {}

# Find rows with the same topics
for i, j in combinations(range(len(topics_lists)), 2):
    if set(topics_lists[i]) == set(topics_lists[j]):
        topic_set = tuple(sorted(set(topics_lists[i])))
        if topic_set in topic_indices:
            topic_indices[topic_set].add(i)
            topic_indices[topic_set].add(j)
        else:
            topic_indices[topic_set] = {i, j}

# Print the indices of rows with the same topics
for indices in topic_indices.values():
    print(list(indices))

[1, 34, 3, 4, 6, 8, 9, 28, 29]
[32, 5]
[10, 77]
[11, 36, 37, 71]
[12, 13]
[40, 15]
[70, 39, 72, 73, 42, 75, 45, 46, 47, 16, 17, 18, 19, 48, 82, 22, 84]
[24, 64, 63, 23]
[25, 61]
[27, 54]
[67, 69, 57, 59, 60, 30, 31]
[81, 44]
[50, 51]
[56, 55]


In [105]:
filtered_df[filtered_df.index.isin([24, 64, 63, 23,27, 54,25, 61])]

,suid,last_update,eqt_code,analyst,date,daclose,topics,headline,body
23,RB0SWET0AFB7,2022-04-28 11:00:00.212,MCD_UN,False,2022-04-28,2022-04-28,BIZNEWS MSCIWORLD NORTHAM CONSD ERN ACEXCLUDE ...,"MCDONALD'S 1Q ADJ EPS $2.28, EST. $2.17.",
24,RB0SYBT0AFBB,2022-04-28 11:00:00.214,MCD_UN,False,2022-04-28,2022-04-28,BIZNEWS MSCIWORLD NORTHAM CONSD ERN ACEXCLUDE ...,"MCDONALD'S 1Q REV. $5.67B, EST. $5.57B.",
25,RB1MZ8DWX2PU,2022-04-28 11:00:00.252,MCD_UN,False,2022-04-28,2022-04-28,METALKEY ALLHOT BIZNEWS MSCIWORLD NORTHAM GLOB...,"MCDONALD'S 1Q COMP SALES +11.8%, EST. +8.73%.",
27,RB0SZBT0AFB6,2022-04-28 11:00:00.253,MCD_UN,False,2022-04-28,2022-04-28,BIZNEWS MSCIWORLD NORTHAM CONSD ERN ACEXCLUDE ...,"MCDONALD'S 1Q US COMP SALES +3.5%, EST. +3.59%.",
54,RFMJX0DWX2PW,2022-07-26 11:00:36.939,MCD_UN,False,2022-07-26,2022-07-26,BIZNEWS MSCIWORLD NORTHAM CONSD ERN ACEXCLUDE ...,"MCDONALD'S 2Q US COMP SALES +3.7%, EST. +2.98%.",
61,RFMK6CBZ8CG0,2022-07-26 11:06:12.985,MCD_UN,False,2022-07-26,2022-07-26,EDGWEB BIZNEWS MSCIWORLD NORTHAM CONSD ACEXCLU...,MCDONALD'S SEES YEAR INTEREST EXPENSE RELATIVE...,
63,RKEO7UDWX2PW,2022-10-27 11:00:00.201,MCD_UN,False,2022-10-27,2022-10-27,BIZNEWS MSCIWORLD NORTHAM CONSD ERN ACEXCLUDE ...,"MCDONALD'S 3Q US COMP SALES +6.1%, EST. +3.7%.",
64,RKDOHDDWX2PT,2022-10-27 11:00:07.305,MCD_UN,False,2022-10-27,2022-10-27,BIZNEWS MSCIWORLD NORTHAM CONSD ERN ACEXCLUDE ...,"MCDONALD'S 3Q ADJ EPS $2.68, EST. $2.57.",


Check the topics that exist in all the news
ACEXCLUDE: excluded from alert catcher -- the news of capital letter 

In [222]:
# Extract the topics from the 'topics' column and split them into lists
topics_lists = filtered_df['topics'].str.split(' ').tolist()

# Find the intersection of topics across all news articles
common_topics = set(topics_lists[0]).intersection(*topics_lists)

# Print the common topics
print("Topics present in all news articles:")
for topic in common_topics:
    print(topic)

Topics present in all news articles:
ACEXCLUDE
G10MEMB
NORTHAM
USMW
MSCIWORLD
MISC
US
RES
SPREGIONS
BIZNEWS
CONSD
BGOVCODES
COS
IL
BGOVBILLGO
SRCRANK1
WORLD
HEADS
PADD2
DEVECO
ALLSTATES
GAMLODRES
CONS
PADDIST
MSCINAMER
EQUITYKEY
G7MEMB
BUSINESS
BONDWIRES
INDUSTRIES


In [223]:
len(common_topics)

30

The percentage of all the other tags

In [224]:
from collections import Counter

# Extract the topics from the 'topics' column and split them into lists
topics_lists = filtered_df['topics'].str.split(' ').tolist()

# Find the intersection of topics across all news articles
common_topics = set(topics_lists[0]).intersection(*topics_lists)

# Remove common topics from topics_lists
topics_lists_without_common = [list(set(topics) - common_topics) for topics in topics_lists]

# Flatten the topics_lists_without_common into a single list
all_topics = [topic for topics in topics_lists_without_common for topic in topics]

# Calculate the percentage of each topic that exists in all news articles
topic_counts = Counter(all_topics)
total_news = len(topics_lists)
topic_percentages = {topic: (count / total_news) * 100 for topic, count in topic_counts.items()}
# Filter out topics with a percentage count less than 5%
filtered_topics = [topic for topic, percentage in topic_percentages.items() if percentage < 5]
# Sort the topics by percentage in descending order
sorted_topics = sorted(topic_percentages.items(), key=lambda x: x[1], reverse=True)

# Print the sorted topics and their percentages
print("Topic\t\tPercentage")
for topic, percentage in sorted_topics:
    print(f"{topic}\t\t{percentage:.2f}%")

Topic		Percentage
FINNEWS		97.65%
ERN		71.76%
RIF		57.65%
HOU		51.76%
CONSS		51.76%
FOD		51.76%
RET		51.76%
RLSHEAD		31.76%
CPNYCNT1		18.82%
EDG		14.12%
FILINGS		14.12%
EDGWEB		14.12%
CFDOCS		14.12%
EDGSDR		14.12%
DEBTKEY		12.94%
RATESKEY		12.94%
FXKEY		12.94%
CREDITKEY		12.94%
CMDKEY		11.76%
ECO		11.76%
CMP		10.59%
INF		9.41%
FIALL		9.41%
ECOINDIC		9.41%
FIASST		9.41%
ECOSTAT		9.41%
CMD		7.06%
CMPPUB		7.06%
GUIDANCE		7.06%
MARKETS		5.88%
EQTY		5.88%
NRGKEY		5.88%
METALKEY		5.88%
CMDMISC		5.88%
CMDX		5.88%
BFWEQ		4.71%
FIRSTWORD		4.71%
INMONEY		3.53%
MAJOR		3.53%
RUNEWS		3.53%
TOP		3.53%
OILKEY		3.53%
ALLHOT		3.53%
INEQUITY		3.53%
MTGEKEY		3.53%
CMDGLOMAC		3.53%
REGIONFILT		3.53%
BROKERLITE		3.53%
TIMECO		3.53%
GLOBALMACR		3.53%
HOT		3.53%
TIMENI		3.53%
AGRKEY		3.53%
POLIRISK		3.53%
GOV		3.53%
LIFESTYLE		3.53%
GDRESTS		3.53%
DINE		3.53%
NASIA		3.53%
ASIAX		3.53%
EXUSSR		3.53%
CMPAUTO		3.53%
HEADGEN		3.53%
BFWUS		2.35%
FIXTURE		2.35%
STKMOV		2.35%
STK		2.35%
ALLMOV		2.35%
BFWFOCUS		2.35

Topic tags, which percentage is less than 5%

In [185]:
print(filtered_topics)


['SSSALES', 'COEVNT', 'BFWEQ', 'FIRSTWORD', 'BFWUS', 'BB', 'MAJOR', 'RUNEWS', 'TOP', 'OILKEY', 'BRNEWS', 'ALLHOT', 'MTGEKEY', 'CMDKEY', 'RATESKEY', 'CMDGLOMAC', 'FXKEY', 'REGIONFILT', 'TIMECO', 'GLOBALMACR', 'MARKETS', 'HOT', 'TIMENI', 'AGRKEY', 'NRGKEY', 'METALKEY', 'BRIC', 'ASIA', 'ESGRES', 'CHINA', 'SUPPLYDISR', 'ESGCONTROV', 'MSCIEMAS', 'NASIA', 'SUPPLY', 'EM', 'ASIAX', 'FRX', 'LIFESTYLE', 'GDRESTS', 'GEN', 'DINE', 'RESTOPEN', 'NOB', 'COR', 'DRA', 'BON', 'CRA', 'FIASST', 'FISUB', 'CORPFIN', 'DIRECTHEAD', 'CBM', 'FIALL', 'BFWTOPFEED', 'TOPBFW', 'ANAESTCHG', 'ANAMOVES', 'ANA', 'ANAESTDOWN', 'CHARGES', 'ANAESTUP', 'TAX', 'POLIRISK', 'GOV', 'BBK', 'CACT', 'STK', 'TNM', 'POSPRE', 'BFW', 'MET', 'CMD', 'BASIC', 'METMNG', 'METSTAT', 'CMDX', 'COMSTAT']


In [186]:
un_meaningful_tags=['FINNEWS','RLSHEAD','ALLSTATES','CPNYCNT1','CMP']

The headline that contains topic tags of a certain threshold 

In [61]:
def create_small_dataframe(df, percentage_low,percentage_up):
    # Sort the topics by percentage in descending order
    sorted_topics = sorted(topic_percentages.items(), key=lambda x: x[1], reverse=True)

    # Create an empty dictionary to store the news headlines and their corresponding topic tags
    small_df_data = {}

    # Iterate over the topics and percentages
    for topic, percentage in sorted_topics:
        # Check if the percentage is less than the threshold
        if percentage_low<percentage < percentage_up:
            # Iterate over the rows of the input dataframe
            for index, row in df.iterrows():
                headline = row['headline']
                topics = row['topics'].split(' ')
                # Check if the topic is in the list of topics for the current row
                if topic in topics:
                    if headline not in small_df_data:
                        small_df_data[headline] = [topic]
                    else:
                        small_df_data[headline].append(topic)

    # Create a list of tuples with the headline and list of topics
    small_df_data_tuples = [(headline, topics) for headline, topics in small_df_data.items()]

    # Create the small dataframe
    small_df = pd.DataFrame(small_df_data_tuples, columns=['headline', 'topics'])

    return small_df


In [115]:
create_small_dataframe(filtered_df, 3,4)

,headline,topics
0,"MCDONALD'S 4Q COMP SALES +12.3%, EST. +10.7%.","[INMONEY, MAJOR, RUNEWS, TOP, OILKEY, ALLHOT, ..."
1,"MCDONALD'S 1Q COMP SALES +11.8%, EST. +8.73%.","[INMONEY, MAJOR, RUNEWS, TOP, OILKEY, ALLHOT, ..."
2,"MCDONALD'S 3Q COMP SALES +9.5%, EST. +5.8%.","[INMONEY, MAJOR, RUNEWS, TOP, OILKEY, ALLHOT, ..."
3,MCDONALD'S SEES YR EFFECTIVE INCOME TAX RATE I...,"[POLIRISK, GOV]"
4,MCD: CHINA 4Q RESULTS HURT BY GOVERNMENT COVID...,"[POLIRISK, GOV, NASIA, ASIAX]"
5,MCD: SEE YR EFFECTIVE TAX RATE 20% TO 22% RANGE.,"[POLIRISK, GOV]"
6,"MCDONALD'S SEES OPENING OVER 1,800 RESTAURANTS...","[LIFESTYLE, GDRESTS, DINE]"
7,"MCD: SEE OPENING ~1,700-1,800 RESTAURANTS GLOB...","[LIFESTYLE, GDRESTS, DINE]"
8,"MCDONALD'S SEES OPENING ~1,700 RESTAURANTS GLO...","[LIFESTYLE, GDRESTS, DINE]"
9,MCDONALD'S: SOME STORES IN CHINA STILL CLOSED ...,"[NASIA, ASIAX]"


In [62]:
def excluded_tag(df, excluded_topic):
    # Create an empty dictionary to store the news headlines and their corresponding topic tags
    small_df_data = {}

    # Iterate over the rows of the input dataframe
    for index, row in df.iterrows():
        headline = row['headline']
        topics = row['topics'].split(' ')
        # Check if the excluded topic is not in the list of topics for the current row
        if excluded_topic not in topics:
            if headline not in small_df_data:
                small_df_data[headline] = topics
            else:
                small_df_data[headline].extend(topics)

    # Create a list of tuples with the headline and list of topics
    small_df_data_tuples = [(headline, topics) for headline, topics in small_df_data.items()]

    # Create the small dataframe
    small_df = pd.DataFrame(small_df_data_tuples, columns=['headline', 'topics'])

    return small_df


In [51]:
excluded_tag(filtered_df, 'FINNEWS')

,headline,topics
0,MCDONALD'S 2Q GLOBAL SAME-STORE SALES DOWN 0.7%.,"[ACEXCLUDE, BGOVBILLGO, BIZNEWS, BONDWIRES, CM..."
1,MCDONALD'S 2Q U.S. SAME-STORE SALES DOWN 2%.,"[ACEXCLUDE, BGOVBILLGO, BIZNEWS, BONDWIRES, CM..."
2,MCDONALD'S 2Q REV. $6.50B.,"[ACEXCLUDE, BGOVBILLGO, BIZNEWS, BONDWIRES, CM..."
3,MCDONALD'S 2Q EPS $1.26.,"[ACEXCLUDE, BGOVBILLGO, BIZNEWS, BONDWIRES, CM..."
4,MCDONALD'S 2Q REV. $6.50B EST. $6.43B.,"[ACEXCLUDE, BGOVBILLGO, BIZNEWS, BONDWIRES, CM..."
5,MCDONALD'S 3Q REV. $6.62B.,"[ACEXCLUDE, ALLSTATES, PADDIST, HEADS, WORLD, ..."
6,MCDONALD'S 3Q EPS $1.40.,"[ACEXCLUDE, ALLSTATES, PADDIST, HEADS, WORLD, ..."
7,MCDONALD'S 3Q REV. $6.62B EST. $6.41B.,"[ACEXCLUDE, ALLSTATES, PADDIST, HEADS, WORLD, ..."
8,MCDONALD’S 3Q REV. $6.62B EST. $6.41B.,"[ACEXCLUDE, ALLSTATES, PADDIST, HEADS, FIRSTWO..."


The headline that contains certain topic tags

In [63]:
def print_headlines_by_topics(df, topics):
    for index, row in df.iterrows():
        headline = row['headline']
        row_topics = row['topics'].split(' ')
        if all(topic in row_topics for topic in topics):
            print(headline)


In [226]:
topic_tags=set(topics_list) - common_topics-set(un_meaningful_tags)-set(filtered_topics)

In [227]:
len(topic_tags)

31

In [228]:
from collections import defaultdict
from collections import Counter

# Create a list containing the topic lists
topic_lists = [list(set(topics) - common_topics-set(un_meaningful_tags)-set(filtered_topics)) for topics in topics_lists]

# Count the occurrences of each topic
topics_count = Counter([topic for topics in topic_lists for topic in topics])

# Create the nested dictionary
topics_dict = defaultdict(dict)

# Iterate over the topic lists
for topic_list in topic_lists:
    # Iterate over the elements in the current topic list
    for main_topic in topic_list:
        # Iterate over the elements in the topic list, excluding the main topic itself
        for sub_topic in topic_list:
            if sub_topic != main_topic:
                topics_dict[main_topic][sub_topic] = topics_dict[main_topic].get(sub_topic, 0) + 1

# Create a list to store the pairs
pairs = []

# Iterate over the keys and sub-dictionaries in topics_dict
for main_topic, sub_dict in topics_dict.items():
    # Iterate over the sub-topics and their counts in the current sub-dictionary
    for sub_topic, count in sub_dict.items():
        # Check if the count of the sub-topic matches the count in topics_count
        if count == topics_count[sub_topic] and topics_count[sub_topic] < topics_count[main_topic]:
            pairs.append((main_topic, sub_topic))

# Create dictionaries to store the sub-topics and main topics
sub_topic_counts = {}
main_topic_dict = {}

# Count the occurrences of sub-topics and store the main topics
for main, sub in pairs:
    sub_topic_counts[sub] = sub_topic_counts.get(sub, 0) + 1
    if sub_topic_counts[sub] == 1:
        main_topic_dict[sub] = main

# Print the main topics and sub-topics
for sub_topic, count in sub_topic_counts.items():
    if count == 1:
        main_topic = main_topic_dict[sub_topic]
        print(f"Main Topic: {main_topic} -> Sub Topic: {sub_topic}")


Main Topic: ERN -> Sub Topic: RIF
Main Topic: ERN -> Sub Topic: HOU
Main Topic: ERN -> Sub Topic: CONSS
Main Topic: ERN -> Sub Topic: RET
Main Topic: ERN -> Sub Topic: FOD
Main Topic: ERN -> Sub Topic: CMPPUB


In [229]:
pairs

[('RIF', 'MARKETS'),
 ('RIF', 'NRGKEY'),
 ('RIF', 'CREDITKEY'),
 ('RIF', 'DEBTKEY'),
 ('RIF', 'METALKEY'),
 ('RIF', 'CMDKEY'),
 ('RIF', 'RATESKEY'),
 ('RIF', 'EQTY'),
 ('RIF', 'FXKEY'),
 ('RIF', 'INF'),
 ('RIF', 'ECO'),
 ('RIF', 'FIALL'),
 ('RIF', 'ECOINDIC'),
 ('RIF', 'FIASST'),
 ('RIF', 'ECOSTAT'),
 ('RIF', 'CMD'),
 ('RIF', 'CMDMISC'),
 ('RIF', 'CMDX'),
 ('CREDITKEY', 'INF'),
 ('CREDITKEY', 'FIALL'),
 ('CREDITKEY', 'ECOINDIC'),
 ('CREDITKEY', 'FIASST'),
 ('CREDITKEY', 'ECOSTAT'),
 ('DEBTKEY', 'INF'),
 ('DEBTKEY', 'FIALL'),
 ('DEBTKEY', 'ECOINDIC'),
 ('DEBTKEY', 'FIASST'),
 ('DEBTKEY', 'ECOSTAT'),
 ('CMDKEY', 'NRGKEY'),
 ('CMDKEY', 'CMD'),
 ('CMDKEY', 'CMDMISC'),
 ('CMDKEY', 'CMDX'),
 ('RATESKEY', 'INF'),
 ('RATESKEY', 'FIALL'),
 ('RATESKEY', 'ECOINDIC'),
 ('RATESKEY', 'FIASST'),
 ('RATESKEY', 'ECOSTAT'),
 ('ERN', 'MARKETS'),
 ('ERN', 'RIF'),
 ('ERN', 'NRGKEY'),
 ('ERN', 'CREDITKEY'),
 ('ERN', 'DEBTKEY'),
 ('ERN', 'METALKEY'),
 ('ERN', 'CMDKEY'),
 ('ERN', 'RATESKEY'),
 ('ERN', 'EQTY')

In [160]:
topics=[]
for topic_list in topics_lists:
    if topic not in topics:
        topics.append(topic)

In [193]:
len(list(set(mains+subs)))

14

In [194]:
len(sub)

5

In [192]:
mains=[]
for main, sub in pairs:
    if main not in mains:
        mains.append(main)
subs=[]
for main, sub in pairs:
    if sub not in subs:
        subs.append(sub)

In [120]:
len(mains)

13

In [179]:
for i in topic_tags:
    if i not in list(set(mains+subs)):
        print(i)


FILINGS
EDG
GUIDANCE
EDGSDR
EDGWEB
CFDOCS


By observation below, RET must go with ERN 

In [122]:
print_headlines_by_topics(filtered_df, ['ERN'])

MCDONALD'S 4Q COMP SALES +12.3%, EST. +10.7%.
MCDONALD'S FALLS 2.9% AFTER 4Q EPS, REVENUE TRAIL ESTIMATES.
MCD CEO: RISING INFLATION IS PUTTING PRESSURE ON RESTAURANTS.
MCDONALD'S CEO CHRIS KEMPCZINSKI SPEAKS ON EARNINGS CALL.
MCDONALD'S: GERMANY, FRANCE RECOVERY HURT BY COVID RESURGENCES.
MCD: OPERATING CONDITIONS STILL CHALLENGING IN GERMANY, FRANCE.
MCD: CHINA 4Q RESULTS HURT BY GOVERNMENT COVID RESTRICTIONS.
MCD SEES 'SIGNIFICANT' COMMODITY, LABOR INFLATION IN NEAR TERM.
MCDONALD'S SEES NET NEW RESTAURANT GROWTH IN U.S. IN 2022.
MCDONALD'S SAYS 1% OF U.S. RESTAURANTS HAVE LIMITED HOURS.
MCDONALD'S SAYS U.S. MARKET STILL FACING STAFFING PROBLEMS.
MCDONALD'S: OMICRON MADE STAFFING MORE DIFFICULT IN U.S.
MCDONALD'S CFO: THERE IS COMMODITY PRESSURE GOING INTO 2022.
MCDONALD'S CEO SEES CONTINUED WAGE PRESSURE THIS YEAR.
MCDONALD'S U.S. DRIVE-THRU TIMES SLOWED IN 2021 VERSUS 2020.
MCDONALD'S 1Q ADJ EPS $2.28, EST. $2.17.
MCDONALD'S 1Q REV. $5.67B, EST. $5.57B.
MCDONALD'S 1Q COMP SALES +1

In [ ]:
filtered_df['topics']=filtered_df['topics'].apply(lambda x: list(set(mains).intersection(set(x.split(' ')))))

In [195]:
filtered_df['intersection'] = filtered_df['topics'].apply(lambda x: list(set(mains).intersection(set(x.split(' ')))))
filtered_df['intersection_sub'] = filtered_df['topics'].apply(lambda x: list(set(subs).intersection(set(x.split(' ')))))
filtered_df['tags'] = filtered_df['topics'].apply(lambda x: list(set(topic_tags).intersection(set(x.split(' ')))))

In [196]:
with open('/mnt/research-live/user/yzhong/MCD_UN_topics_2015.txt', 'w') as f:
    for row in filtered_df.itertuples():
        f.write(str(row.tags) + '\n')

In [204]:
list1=filtered_df[filtered_df['headline']=="MCDONALD'S 3Q REV. $6.62B EST. $6.41B."].topics.str.split(' ').sum()
intersection = set(mains).intersection(list1)

### The main topics

In [26]:
mains=[]
for main, sub in pairs:
    if main not in mains:
        mains.append(main)
subs=[]
for main, sub in pairs:
    if sub not in subs:
        subs.append(sub)

In [28]:
len(mains)

43

In [27]:
subs=[]
for main, sub in pairs:
    if sub not in subs:
        subs.append(sub)

In [208]:
len(subs)

95

In [251]:
print_headlines_by_topics(filtered_df, ['ERN','FINNEWS'])

MCDONALD'S 4Q GLOBAL SAME-STORE SALES DOWN 0.9%.
MCDONALD'S 4Q REV. $6.57B.
MCDONALD'S 4Q EPS $1.13.
MCDONALD'S 4Q REV. $6.57B EST. $6.67B.
MCDONALD’S 4Q REV. $6.57B EST. $6.67B.
MCDONALD'S 4Q GLOBAL COMP SALES DOWN 1%.
MCDONALD'S 4Q GLOBAL COMP SALES DOWN 1.0%,.
MCDONALD'S SEES JAN. COMP SALES TO BE NEGATIVE.
MCDONALD'S 4Q GLOBAL COMP SALES DOWN 1% EST. DOWN. 1.5%.
MCDONALD'S 4Q GLOBAL COMP SALES DOWN 1.0%, EST. DOWN 1.5%.
MCDONALD'S 2015 CAPITAL SPENDING PLAN $2B.
MCDONALD'S 4Q GLOBAL COMP SALES DOWN 0.9%.
MCDONALD'S 4Q GLOBAL COMP SALES DOWN 0.9% EST. DOWN 1.5%.
MCDONALD'S 2015 CAPITAL SPENDING PLAN, LOWEST IN OVER 5.
MCDONALD'S 4Q EPS $1.13, EST. $1.22.
CORRECT: MCDONALD'S 4Q GLOBAL COMP SALES DOWN 0.9%.
CORRECT: MCDONALD'S 4Q GLOBAL COMP DOWN 0.9%, EST. DOWN 1.5%.
MCD SEES RESULTS TO REMAIN PRESSURED, PARTICULARLY IN 1H.
MCDONALD'S TARGETING FEWER OPENINGS IN MOST CHALLENGED MARKETS.
MCD TAKING `DECISIVE ACTION' TO REGAIN MOMENTUM IN SALES.
MCDONALD'S 4Q U.S. COMP SALES DOWN 1.7%,

In [250]:
print_headlines_by_topics(filtered_df, ['ERN','HEADGEN'])

MCDONALD'S 4Q GLOBAL SAME-STORE SALES DOWN 0.9%.
MCDONALD'S 4Q REV. $6.57B.
MCDONALD'S 4Q EPS $1.13.
MCDONALD'S 4Q REV. $6.57B EST. $6.67B.
MCDONALD’S 4Q REV. $6.57B EST. $6.67B.
MCDONALD'S 1Q GLOBAL SAME-STORE SALES DOWN 2.3%.
MCDONALD'S 1Q REV. $5.96B.
MCDONALD'S 1Q EPS 84C.
MCDONALD'S 1Q REV. $5.96B EST. $5.95B.
MCDONALD’S 1Q REV. $5.96B EST. $5.95B.
MCDONALD'S 2Q GLOBAL SAME-STORE SALES DOWN 0.7%.
MCDONALD'S 2Q U.S. SAME-STORE SALES DOWN 2%.
MCDONALD'S 2Q REV. $6.50B.
MCDONALD'S 2Q EPS $1.26.
MCDONALD'S 2Q REV. $6.50B EST. $6.43B.
MCDONALD'S 3Q REV. $6.62B.
MCDONALD'S 3Q EPS $1.40.
MCDONALD'S 3Q REV. $6.62B EST. $6.41B.
MCDONALD’S 3Q REV. $6.62B EST. $6.41B.


In [181]:
print_headlines_by_topics(filtered_df, ['EST'])

MCDONALD'S 1Q INCL CHARGES 17C.
MCDONALD'S 1Q INCLUDES STRATEGIC CHARGES OF $0.17/SHR.
MCDONALD'S 1Q INCL CHARGES 17C, FX NEGATIVE EFFECT 9C.
MCD 1Q GLOBAL COMP SALES DECREASE OF 2.3%,, EST. DOWN 1.8%.
MCDONALD'S SES APRIL GLOBAL COMP SALES NEGATIVE.
MCDONALD'S APRIL GLOBAL COMP SALES ARE EXPECTED TO BE NEGATIVE.
MCDONALD'S SEES APRIL GLOBAL COMP SALES TO BE NEGATIVE.
MCD 1Q U.S. COMP SALES FELL 2.6%, EST. DOWN 2.1%.
MCDONALD'S 1Q EPS EX STRATEGIC CHARGES $1.01.
MCD 1Q EUROPE COMP SALES DECLINED 0.6%, EST. DOWN 0.2%.
MCDONALD'S 1Q EPS EX CHARGES $1.01, EST. $1.06.
MCD 1Q APMEA COMP SALES FELL 8.3%, EST. DOWN 6.5%.
MCDONALD'S TO SHARE DETAILS OF TURNAROUND PLAN MAY 4.
MCDONALD'S 1Q EPS EX CHARGES $1.01, EST. $1.06.
MCD DEVELOPING TURNAROUND PLAN, INITIAL PLAN DETAILS ON MAY 4.
MCDONALD'S 2Q GLOBAL COMP SALES DECREASE OF 0.7%,EST. DOWN 0.4%.
MCDONALD’S 2Q GLOBAL COMP SALES DECREASE OF 0.7%,EST. DOWN 0.4%.
MCDONALD'S 2Q EUROPE COMP SALES INCREASED 1.2%, EST. UP 1.5%.


### The sub topics that must appear together

In [18]:
len(topic_lists)

110

In [49]:
create_small_dataframe(filtered_df, 90,100)

,headline,topics
0,MCDONALD'S 4Q GLOBAL SAME-STORE SALES DOWN 0.9%.,[FINNEWS]
1,MCDONALD'S 4Q REV. $6.57B.,[FINNEWS]
2,MCDONALD'S 4Q EPS $1.13.,[FINNEWS]
3,MCDONALD'S 4Q REV. $6.57B EST. $6.67B.,[FINNEWS]
4,MCDONALD’S 4Q REV. $6.57B EST. $6.67B.,[FINNEWS]
...,...,...
96,"MCDONALD'S 3Q U.S. COMP SALES UP 0.9%, EST. DO...",[FINNEWS]
97,MCDONALD'S 3Q EPS $1.40 WITH 17C FX TRANSLATIO...,[FINNEWS]
98,"MCDONALD'S 3Q INTL LEAD MARKETS COMPS UP 4.6%,...",[FINNEWS]
99,MCDONALD'S 3Q HIGH GROWTH MARKETS COMPS UP 8.9...,[FINNEWS]
